In [25]:
import numpy as np
import pickle as pl
import math as math
import jax as jax
import jax.numpy as jnp
from enum import Enum

In [38]:
def VecRotationMatrix(vect, angle, isRad = False):
    angle = angle if isRad else angle * math.pi / 180
    mag = np.linalg.norm(vect)
    l = vect[0, 0] / mag 
    m = vect[0, 1] / mag
    n = vect[0, 2] / mag
    cos = math.cos(angle)
    omc = 1 - cos
    sin = math.sin(angle)
    return np.matrix([
        [l * l * omc + cos, m * l * omc - n * sin, n * l * omc + m * sin, 0],
        [l * m * omc + n * sin, m * m * omc + cos, m * n * omc - l * sin, 0],
        [l * n * omc - m * sin, m * n * omc + l * sin, n * n * omc + cos, 0],
        [0, 0, 0, 1]])

def VecTranslationMatrix(vect):
    return np.matrix([
        [1, 0, 0, vect[0, 0]],
        [0, 1, 0, vect[0, 1]],
        [0, 0, 1, vect[0, 2]],
        [0, 0, 0, 1]])

def SphericalRotationMatrix(theta, phi, isRad = False):
    thetaRotationMatrix = VecRotationMatrix(np.matrix([0, 0, 1]), theta, isRad)
    phiRotationVector = thetaRotationMatrix * np.matrix([[0], [-1], [0], [1]])

    phiRotationMatrix = VecRotationMatrix( \
        np.matrix([ \
            phiRotationVector[0, 0], \
            phiRotationVector[1, 0], \
            phiRotationVector[2, 0]]), \
        phi, \
        isRad)

    return phiRotationMatrix * thetaRotationMatrix

origin = np.matrix([[0], [0], [0], [1]])
xunit = np.matrix([[1], [0], [0], [1]])
yunit = np.matrix([[0], [1], [0], [1]])
zunit = np.matrix([[0], [0], [1], [1]])
mxunit = np.matrix([[-1], [0], [0], [1]])
myunit = np.matrix([[0], [-1], [0], [1]])
mzunit = np.matrix([[0], [0], [-1], [1]])

In [48]:
transforms = []
for x in [0, 120, 240]:
    transforms.append(
        VecTranslationMatrix(np.matrix([-.77, 0, 0])) * \
        VecRotationMatrix(np.matrix([1, 0, 0]), x) * \
        VecRotationMatrix(np.matrix([0, 0, 1]), 109.1) * \
        VecTranslationMatrix(np.matrix([1.09, 0, 0])) * \
        np.matrix([[0], [0], [0], [1]]))

transforms

[matrix([[-1.12666751],
         [ 1.02999431],
         [ 0.        ],
         [ 1.        ]]),
 matrix([[-1.12666751],
         [-0.51499716],
         [ 0.89200124],
         [ 1.        ]]),
 matrix([[-1.12666751],
         [-0.51499716],
         [-0.89200124],
         [ 1.        ]])]

In [6]:
VecTranslationMatrix(np.matrix([1, 0, 0])) * origin

matrix([[0.99962422],
        [0.02741213],
        [0.        ],
        [1.        ]])

In [39]:
SphericalRotationMatrix(90, 0) * xunit

matrix([[6.123234e-17],
        [1.000000e+00],
        [0.000000e+00],
        [1.000000e+00]])

In [42]:
(VecRotationMatrix(np.matrix([0, 0, 1]), 90) * myunit)[1, 0]

-6.123233995736766e-17

In [37]:
VecRotationMatrix(np.matrix([0, 0, 1]), 90)

matrix([[ 6.123234e-17, -1.000000e+00,  0.000000e+00,  0.000000e+00],
        [ 1.000000e+00,  6.123234e-17,  0.000000e+00,  0.000000e+00],
        [ 0.000000e+00,  0.000000e+00,  1.000000e+00,  0.000000e+00],
        [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]])

In [23]:
VecRotationMatrix(np.matrix([1, 0, 0]), 0) * VecRotationMatrix(np.matrix([0, 0, 1]), 90) * xunit

matrix([[6.123234e-17],
        [1.000000e+00],
        [0.000000e+00],
        [1.000000e+00]])

In [36]:
VecRotationMatrix(np.matrix([1, 0, 0]), 0) * yunit

matrix([[0.],
        [1.],
        [0.],
        [1.]])

matrix([[0],
        [1],
        [0],
        [1]])

In [27]:
xunit

matrix([[1],
        [0],
        [0],
        [1]])

In [28]:
myunit

matrix([[ 0],
        [-1],
        [ 0],
        [ 1]])

In [10]:
x = np.matrix([[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 0, 1], [2, 3, 4, 5]])
y = x[[0, 2]]
y

matrix([[0, 1, 2, 3],
        [8, 9, 0, 1]])

In [11]:
x[np.matrix([0, 1])]

matrix([[[0, 1, 2, 3],
         [4, 5, 6, 7]]])

In [12]:
x[np.matrix([[0], [2]])]

matrix([[[0, 1, 2, 3]],

        [[8, 9, 0, 1]]])

In [13]:
x[np.matrix([[0, 1], [0, 2]])]

matrix([[[0, 1, 2, 3],
         [4, 5, 6, 7]],

        [[0, 1, 2, 3],
         [8, 9, 0, 1]]])

In [14]:
x[0]

matrix([[0, 1, 2, 3]])

In [48]:
# square of distance magnitude

pos = np.matrix([[0, 0], [2, 2], [5, 0]])
edgeI = pos[[0, 1]]
edgeJ = pos[[1, 2]]

dis = edgeI - edgeJ

np.sum(np.square(dis), axis = 1)

matrix([[ 8],
        [13]], dtype=int32)

In [72]:
# dot product

angleI = pos[[0, 2]]
angleJ = pos[[1, 0]]
angleK = pos[[2, 1]]

vect1 = angleI - angleJ
vect2 = angleK - angleJ

dotProd = (vect1 * vect2.transpose()).diagonal().transpose()    # same as np.sum(np.multiply(vect1, vect2), axis = 1) (MORE EFFICIENT)
                                                                # also same as np.dot(vect1, vect2.transpose()).diagonal().transpose() (NEEDS MORE TESTING TO MAKE SURE THIS IS EQUIVALENT)

(vect1, vect2, dotProd, np.sum(np.multiply(vect1, vect2), axis = 1), np.dot(vect1, vect2.transpose()).diagonal().transpose())

(matrix([[-2, -2],
         [ 5,  0]]),
 matrix([[ 3, -2],
         [ 2,  2]]),
 matrix([[-2],
         [10]]),
 matrix([[-2],
         [10]]),
 matrix([[-2],
         [10]]))

In [74]:
# vector angle

mag1 = np.sqrt(np.sum(np.square(vect1), axis = 1))
mag2 = np.sqrt(np.sum(np.square(vect2), axis = 1))
magProd = (mag1 * mag2.transpose()).diagonal().transpose() # same as np.sum(np.multiply(mag1, mag2), axis = 1) (MORE EFFICIENT)

(mag1, mag2, magProd, np.sum(np.multiply(mag1, mag2), axis = 1), dotProd, np.arccos(dotProd / magProd))

(matrix([[2.82842712],
         [5.        ]]),
 matrix([[3.60555128],
         [2.82842712]]),
 matrix([[10.19803903],
         [14.14213562]]),
 matrix([[10.19803903],
         [14.14213562]]),
 matrix([[-2],
         [10]]),
 matrix([[1.76819189],
         [0.78539816]]))

In [75]:
# cross product

vect3 = np.matrix([[1, 2, 3], [4, 5, 6]])
vect4 = np.matrix([[4, 5, 6], [1, 2, 3]])

(vect3, vect4, np.cross(vect3, vect4))

(matrix([[1, 2, 3],
         [4, 5, 6]]),
 matrix([[4, 5, 6],
         [1, 2, 3]]),
 array([[-3,  6, -3],
        [ 3, -6,  3]]))

In [78]:
# torsional angle
[(i, k, v) for i, (k, v) in enumerate(({"foo": 10, "bar": 11}).items())]


[(0, 'foo', 10), (1, 'bar', 11)]

In [41]:
def angle_2(R, nplib):
    r1 = R[0,:]-R[1,:]
    r2 = R[2,:]-R[1,:]
    dot = nplib.sum(nplib.multiply(r1, r2))
    r1_mag = nplib.sqrt(nplib.sum(nplib.square(r1)))
    r2_mag = nplib.sqrt(nplib.sum(nplib.square(r2)))
    angle = nplib.arccos(dot / (r1_mag * r2_mag))
    return angle
def angle_np(R):
    return angle_2(R, np)

result = angle_np(np.array([
    [-1, 2, 1],
    [3, 3, 1],
    [5, 1, -1]
    ]))

print(result)

2.0043342361507333


In [42]:
def angle_jnp(R):
    return angle_2(R, jnp)

angle = jax.vmap(angle_jnp, in_axes = (0, ))

params = jnp.array([
    [[-1, 2, 1],
    [3, 3, 1],
    [5, 1, -1]],
    [[-1, 2, 1],
    [3, 3, 1],
    [5, 1, -1]],
    [[-1, 2, 1],
    [3, 3, 1],
    [5, 1, -1]],
])
result = angle(params)

print(result)

[2.0043342 1.7331997 1.3984457]


In [31]:
jax.numpy.ma

DeviceArray([[0.7551559 , 0.3129729 , 0.12388372],
             [0.548188  , 0.8851279 , 0.30576992],
             [0.82008433, 0.95633745, 0.3566252 ]], dtype=float32)